In [138]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col, isnan, when, count, desc, collect_set

In [139]:
# SparkSession 생성
spark = SparkSession.builder \
    .appName("HDFS File Read Example") \
    .getOrCreate()

In [140]:
sellers = spark.read.csv("hdfs://master:9000/raw/olist/olist_sellers_dataset.csv", header=True, inferSchema=True)
order_items = spark.read.csv("hdfs://master:9000/raw/olist/olist_order_items_dataset.csv", header=True, inferSchema=True)
orders = spark.read.csv("hdfs://master:9000/raw/olist/olist_orders_dataset.csv", header=True, inferSchema=True)
products = spark.read.csv("hdfs://master:9000/raw/olist/olist_products_dataset.csv", header=True, inferSchema=True)
products_eng = spark.read.csv('hdfs://master:9000/preprocessed/olist/products_eng_updated_single.csv', header=True, inferSchema=True)


In [146]:
# from pyspark.sql.functions import explode

# order_items.createOrReplaceTempView("order_items")
# # 예: seller_id, products (array 형태) 가 있는 DataFrame
# df = spark.sql("""
# SELECT 
#   seller_id, 
#   COLLECT_SET(product_id) AS products
# FROM order_items
# GROUP BY seller_id
# """)

# # # products 배열을 행으로 펼치기
# # flattened = df.select("seller_id", explode("products").alias("product_id"))
# # flattened.show()

# spark.sql("""
# SELECT seller_id, product_id
# FROM (
#   SELECT seller_id, COLLECT_SET(product_id) AS products
#   FROM order_items
#   GROUP BY seller_id
# ) tmp
# LATERAL VIEW explode(products) t AS product_id
# """).show()


In [142]:
unique_pairs = order_items.select("seller_id", "product_id").distinct()
unique_pairs.show()

+--------------------+--------------------+
|           seller_id|          product_id|
+--------------------+--------------------+
|1da3aeb70d7989d1e...|efec0b0b076ce3f16...|
|620c87c171fb2a6dd...|e9eebb8e8ba0fadb9...|
|18a349e75d307f4b4...|28beefcab28e6a3b6...|
|6edacfd9f9074789d...|b2572a992f6ea1d1a...|
|c3cfdc648177fdbbb...|cbd2c1ba72b7f4ce3...|
|6560211a19b47992c...|16bf176650a888512...|
|3d8fa2f5b647373c8...|d339b523c298009f1...|
|004c9cd9d87a3c30c...|daf9301b3e3756d45...|
|1bb2bdb95f4841f1b...|0d368e3a4bd2ebf60...|
|de23c3b98a8888828...|84a5ff7347e4371b2...|
|845e8036c3f7bf620...|8ebb1bc470f9bfd40...|
|1b65c144b17e607c0...|638a9114d65182d28...|
|955fee9216a65b617...|c20a3f598c16d7724...|
|ac3508719a1d8f5b7...|1a366f773e013eb49...|
|530ec6109d11eaaf8...|008cff0e5792219fa...|
|530ec6109d11eaaf8...|438becbce7afb5a88...|
|4c2b230173bb36f9b...|580cb7f00ab0b9491...|
|05ff92fedb5be4792...|830345ab5d2e365c7...|
|54a1852d1b8f10312...|2b0c892ab2b8e600a...|
|7aa4334be125fcdd2...|64315bd8c0

In [143]:
product_lists = unique_pairs.groupBy("seller_id").agg(collect_set("product_id").alias("unique_product_ids"))
product_lists.show(truncate=False)

+--------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [135]:
order_items.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [ ]:
order_items.groupBy(["seller_id", 'product_id']).count().orderBy('seller_id').show()

+--------------------+--------------------+-----+
|           seller_id|          product_id|count|
+--------------------+--------------------+-----+
|0015a82c2db000af6...|a2ff5a97bf95719e3...|    3|
|001cca7ae9ae17fb1...|e251ebd2858be1aa7...|   48|
|001cca7ae9ae17fb1...|4d7fee7877228c149...|    2|
|001cca7ae9ae17fb1...|4f3b83b83f7fb280f...|    1|
|001cca7ae9ae17fb1...|98a8c2fa16d7239c6...|   30|
|001cca7ae9ae17fb1...|21fecd254a3103704...|    3|
|001cca7ae9ae17fb1...|0da9ffd92214425d8...|   17|
|001cca7ae9ae17fb1...|86b22a03cb72239dd...|    6|
|001cca7ae9ae17fb1...|547b95702aec86f05...|   17|
|001cca7ae9ae17fb1...|08574b074924071f4...|  113|
|001cca7ae9ae17fb1...|d32f22e03fb01595a...|    1|
|001cca7ae9ae17fb1...|6d15a14a5c04e3ef3...|    1|
|001e6ad469a905060...|093cd981b714bcdff...|    1|
|002100f778ceb8431...|e619ceb6b2a419677...|    1|
|002100f778ceb8431...|54151de8a5ccf2dc3...|    1|
|002100f778ceb8431...|f5765f6aff9701523...|    1|
|002100f778ceb8431...|43840498716c5a018...|    1|


In [108]:
order_counts_by_seller = order_items.groupBy("seller_id").count().orderBy('count', ascending=False)
order_counts_by_seller.show()

+--------------------+-----+
|           seller_id|count|
+--------------------+-----+
|6560211a19b47992c...| 2033|
|4a3ca9315b744ce9f...| 1987|
|1f50f920176fa81da...| 1931|
|cc419e0650a3c5ba7...| 1775|
|da8622b14eb17ae28...| 1551|
|955fee9216a65b617...| 1499|
|1025f0e2d44d7041d...| 1428|
|7c67e1448b00f6e96...| 1364|
|ea8482cd71df3c196...| 1203|
|7a67c85e85bb2ce85...| 1171|
|4869f7a5dfa277a7d...| 1156|
|3d871de0142ce09b7...| 1147|
|8b321bb669392f516...| 1018|
|cca3071e3e9bb7d12...|  830|
|620c87c171fb2a6dd...|  798|
|a1043bafd471dff53...|  770|
|e9779976487b77c6d...|  750|
|f8db351d8c4c4c22c...|  724|
|d2374cbcbb3ca4ab1...|  631|
|391fc6631aebcf300...|  613|
+--------------------+-----+
only showing top 20 rows



In [ ]:
top_seller_id = order_counts_by_seller.select("seller_id").first()[0]
top_seller_orders = order_items.filter(order_items.seller_id == top_seller_id)
top_seller_orders.show()

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|001021efaa8636c29...|            1|5d7c23067ed3fc8c6...|6560211a19b47992c...|2018-03-05 09:35:41| 49.0|         15.1|
|00418a49a685c6bb6...|            1|5e21d5cab5d33e770...|6560211a19b47992c...|2017-10-09 20:50:45| 49.0|         7.78|
|005a1dded353107db...|            1|7e6c4a0bf900e259f...|6560211a19b47992c...|2018-02-15 18:10:50| 75.0|        14.28|
|006e05e81c068ddc3...|            1|02475368dfb38934f...|6560211a19b47992c...|2018-01-03 22:07:25| 29.0|         14.1|
|00750f68bb6453703...|            1|526f833e1eb0a32ad...|6560211a19b47992c...|2017-11-17 10:10:32|129.0|         8.16|
|008819c6d5f6da6fa...|            1|57f2bc497c1a

In [115]:
top_seller_orders.select(top_seller_orders.order_item_id).distinct().show()

+-------------+
|order_item_id|
+-------------+
|            1|
|            6|
|            3|
|            5|
|            4|
|            2|
+-------------+



In [121]:
top_order_id = top_seller_orders.filter(top_seller_orders.order_item_id == 6).select('order_id').first()[0]
top_order_id

'2c52a5a630ca9e849ec5fad8932bab44'

In [123]:
top_product_id = order_items.filter(order_items.order_id == top_order_id).select('product_id').first()[0]
top_product_id

'7703b8cb5b77fbb36518ea91e2ca734f'

In [126]:
top_product_id_cateory_name = products.filter(col('product_id') == top_product_id).select('product_category_name').first()[0]

In [127]:
products_eng.filter(col('product_category_name') == top_product_id_cateory_name).select('product_category_name_english').show()

+-----------------------------+
|product_category_name_english|
+-----------------------------+
|                watches_gifts|
+-----------------------------+

